# Code for interacting with API of openFDA

## Plan
In this project, we are looking for device adverse events. We are particularly interested in:
    - Cause of failure
    - Date of failure (ie age of device)
We want to see which is the most common cause for given device to fail and at which stage of use this occurs.
Relevant fields could be (for full reference see https://open.fda.gov/device/event/reference/):
### Event
device_date_of_manufacturer
date_of_event, date_report, date_received
previous_use_code, remedial_action
single_use_flag
### Source
reprocessed_and_reused_flag
### Device
device.generic_name
device.expiration_date_of_device, device.device_age_text
device.implant_flag, device.date_removed_flag
device.manufacturer_d_name, device.manufacturer_d_state, device.manufacturer_d_country
### Patient
patient.sequence_number_outcome, patient.sequence_number_treatment
### Report Text
mdr_text.text, mdr_text.text_type_code
### Reporter Dependent Fields
#### By user facility / importer
report_date
event_location
manufacturer_name, manufacturer_country
manufacturer_g1_name, manufacturer_g1_state
### OpenFDA fields
device_class
### Further interesting fields:
Source: reporter_occupation_code
Device: device.device_operator

In [2]:
import numpy as np
import pandas as pd
import json
import requests

fileName = 'device-event-2016Q1-JSON~\device-event-0001-of-0002.json'
baseurl = 'https://api.fda.gov/device/event.json'

In [26]:
# Example of quering, for complete guide go to: https://open.fda.gov/api/
q1 = baseurl + '?search=device.generic_name:"pump"&limit=5'
dq1 = requests.get(q1)
# dq1.json()['results']
data = json.loads(dq1.text)
results = data['results']

In [27]:
# Fields of Interest
fois_device = ['generic_name', 'expiration_date_of_device', 'device_age_text', 'implant_flag', 'date_removed_flag', \
               'manufacturer_d_name', 'manufacturer_d_state', 'manufacturer_d_country']
fois_patient = ['sequence_number_outcome', 'sequence_number_treatment']
fois_mdrText = ['text', 'text_type_code']
fois_openfda = ['device_name', 'device_class', 'medical_specialty_description']

# device = data['results'][0]['device'][0]
device = [x['device'][0] for x in data['results']]
# patient = data['results'][0]['patient'][0]
patient = [x['patient'][0] for x in data['results']]
# mdrText = data['results'][0]['mdr_text'][0] # there may be more items in the list! 
mdrText = [x['mdr_text'][0] for x in data['results']]
# openfda = data['results'][0]['device'][0]['openfda']
openfda = [x['device'][0]['openfda'] for x in data['results']]

In [28]:
df_openfda = pd.DataFrame(openfda, index = range(len(results)),columns = fois_openfda)
df_device = pd.DataFrame(device, index = range(len(results)),columns = fois_device)
df_patient = pd.DataFrame(patient, index = range(len(results)),columns = fois_patient)
df_mdrText = pd.DataFrame(mdrText, index = range(len(results)),columns = fois_mdrText)
df = pd.concat([df_device, df_patient, df_mdrText, df_openfda], axis = 1)
df

,generic_name,expiration_date_of_device,device_age_text,implant_flag,date_removed_flag,manufacturer_d_name,manufacturer_d_state,manufacturer_d_country,sequence_number_outcome,sequence_number_treatment,text,text_type_code,device_name,device_class,medical_specialty_description
0,PUMP,NaN,,N,B,,,,[],[],"PATIENT CALLED, HAS HX OF TIDM, 5 MONTHS PREGN...",Description of Event or Problem,"Pump, Infusion, Insulin",2,General Hospital
1,PARADIGM 722 INSULIN PUMP,NaN,,N,B,,CA,US,[],[],PATIENT PRESENTED TO ENDOCRINOLOGY OFFICE WITH...,Description of Event or Problem,"Pump, Infusion, Insulin",2,General Hospital
2,INFUSION PUMP - PATIENT CONTROLLED,NaN,,N,*,BAXTER HEALTHCARE CORP. ELECTRONIC INFUSION SYS.,IL,US,[1. O],[],PT WAS PUT ON MORPHINE TO BE ADMINISTERED BY P...,Description of Event or Problem,"Pump, Infusion, Pca",2,General Hospital
3,INSULIN INFUSION PUMP,NaN,DA,N,B,ANIMAS CORP.,PA,US,[1. H],[],THE PATIENT WAS HOSPITALIZED FOR HYPOGLYCEMIA ...,Description of Event or Problem,"Pump, Infusion, Insulin",2,General Hospital
4,INSULIN INFUSION PUMP,NaN,DA,N,B,ANIMAS CORP.,PA,US,[1. H],[],THE PATIENT WAS HOSPITALIZED FOR ELEVATED BLOO...,Description of Event or Problem,"Pump, Infusion, Insulin",2,General Hospital
